In [8]:
# let's try to run LSTM on RSRP values from ns-3 dual strip simulation

import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split


In [9]:
# First, read and prepare RSRP data
file = 'DlRsrpSinrStats_hom-0_ttt-64.txt'
h = open(file, 'r')
hlines = h.readlines()

base = []
for line in hlines:
    base.append(line.split())

# Organize data frame
base = pd.DataFrame(base)

base.shape

(195970, 8)

In [10]:
# transform RSRP from linear to dB

base.drop(columns=[1, 3, 6, 7], inplace=True)

base.columns=['time', 'IMSI', 'rsrp', 'sinr']

base = base.iloc[1:]

base['rsrp'] = np.log10(base['rsrp'].values.astype(float))*10

base.shape

(195969, 4)

In [11]:
# get only RSRP values from 1 UE as time series
myrsrp = []
myrsrp = base.loc[base['IMSI'].astype(int)==12, 'rsrp']
myrsrp.reset_index(drop=True, inplace=True)
myrsrp = pd.DataFrame(myrsrp).values

myrsrp.shape

(9800, 1)

In [12]:
# apply MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
myrsrp_norm = scaler.fit_transform(myrsrp)

# train and test split 

rsrptrain, rsrptest = train_test_split(
    myrsrp_norm, test_size=0.9, random_state=42
    )

In [13]:
# Training phase
# initialing variables

prev = []
real_rsrp = []

# filling for 100-sample prediction
for i in range(100, rsrptrain.size):
    prev.append(rsrptrain[i-100:i, 0])
    real_rsrp.append(rsrptrain[i, 0])

# adapting formats (only 1 dimension)
prev, real_rsrp = np.array(prev), np.array(real_rsrp)

print('prev.shape',prev.shape)
print('real_rsrp.shape',real_rsrp.shape)
print("reshape..")

prev = np.reshape(prev, (prev.shape[0], prev.shape[1], 1))
print('prev.shape',prev.shape)


prev.shape (880, 100)
real_rsrp.shape (880,)
reshape..
prev.shape (880, 100, 1)


In [14]:
# starting regressor
regressor = Sequential()
regressor.add(LSTM(units = 120, return_sequences = True, input_shape = (prev.shape[1], 1)))

# using dropout to avoid overfitting
regressor.add(Dropout(0.3))

# more layers
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3))

# more layers
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3))

# more layers
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.3))

# final layer
regressor.add(Dense(units = 1, activation = 'linear'))

# compiling
regressor.compile(optimizer = 'rmsprop', loss = 'mean_squared_error', metrics = ['mean_absolute_error'])
regressor.fit(prev, real_rsrp, epochs = 25, batch_size = 32)

# testing phase
# preparing inputs for test
inputs = myrsrp_norm[len(myrsrp_norm) - len(rsrptest) - 100:]
inputs = inputs.reshape(-1, 1)

# loop for filling variable
x_test = []
for i in range (100, inputs.size):
    x_test.append(inputs[i-100:i, 0])

# format adapting
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

prediction = regressor.predict(x_test)

# undo normalization for better viewing our results
prediction = scaler.inverse_transform(prediction)

# get real RSRP test values to plot and compare
real_rsrp_test = myrsrp[8821:9800, :]
mae = mean_absolute_error(real_rsrp_test, prediction)

# visualization
plt.plot(real_rsrp_test, color = 'red', label = 'Real RSRP')
plt.plot(prediction, color = 'blue', label = 'Prediction')
plt.title('RSRP values prediction')
plt.xlabel('Time (samples)')
plt.ylabel('RSRP (dB)')
plt.legend()
plt.grid()
plt.show()

2022-03-16 14:16:38.395047: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-16 14:16:38.397420: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-03-16 14:16:39.398832: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25
28/28 [==============================] - 22s 449ms/step - loss: 0.0540 - mean_absolute_error: 0.1795
Epoch 2/25
28/28 [==============================] - 12s 435ms/step - loss: 0.0330 - mean_absolute_error: 0.1462
Epoch 3/25
28/28 [==============================] - 12s 440ms/step - loss: 0.0318 - mean_absolute_error: 0.1449
Epoch 4/25
28/28 [==============================] - 13s 466ms/step - loss: 0.0292 - mean_absolute_error: 0.1358
Epoch 5/25
28/28 [==============================] - 15s 534ms/step - loss: 0.0263 - mean_absolute_error: 0.1311
Epoch 6/25
28/28 [==============================] - 14s 507ms/step - loss: 0.0266 - mean_absolute_error: 0.1343
Epoch 7/25
28/28 [==============================] - 16s 565ms/step - loss: 0.0255 - mean_absolute_error: 0.1289
Epoch 8/25
28/28 [==============================] - 15s 530ms/step - loss: 0.0259 - mean_absolute_error: 0.1298
Epoch 9/25
28/28 [==============================] - 14s 498ms/step - loss: 0.0265 - mean_absolute_error:

ValueError: Found input variables with inconsistent numbers of samples: [979, 8820]